In [3]:
print('eventSource->', lambdaData['Records'][0]['eventSource'])
print('aws region->', lambdaData['Records'][0]['awsRegion'])
print('objectCreated->', lambdaData['Records'][0]['eventName'])


eventSource-> aws:s3
aws region-> ap-south-1
objectCreated-> ObjectCreated:Put


In [ ]:
import boto3
import urllib.parse


AWS_ACCESS_KEY = ''
AWS_SECRET_KEY = ''
REGION = 'ap-south-1'
BUCKET_NAME = ''
OBJECT_KEY = 'sbi mine.txt'  

s3 = boto3.client('s3',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=REGION
)

def read_s3_file(bucket, key):
    try:
        key = urllib.parse.unquote_plus(key, encoding='utf-8')
        response = s3.get_object(Bucket=bucket, Key=key)
        body = response['Body'].read()
        content = body.decode('utf-8')
        print("File Content:\n", content)
        return content
    except Exception as e:
        print(f"Error accessing object '{key}' from bucket '{bucket}': {e}")
        return None


if __name__ == "__main__":
    read_s3_file(BUCKET_NAME, OBJECT_KEY)

In [ ]:
import csv
import json
import boto3
import time
from datetime import datetime


def lambda_handler(event, context):
    
    event_params=event["Records"][0]
    
    bucket=event_params["s3"]["bucket"]["name"]
    key=event_params["s3"]["object"]["key"]
    
    print("here")
    
    s3_resource = boto3.resource('s3')
    s3_object = s3_resource.Object(bucket, key)
    
    data = s3_object.get()['Body'].read().decode('utf-8').splitlines()
    
    lines = csv.reader(data)
    print(lines)
    headers = next(lines)
    print('headers: %s' %(headers))
    
    list_data = list(lines)
    
    print(list_data)
    india=[]
    us=[]
    for i in list_data:
        if i[3]=='India':
            india.append(int(i[2]))
        else:
            us.append(int(i[2]))
        
    print('total india salary spend is ',sum(india))
    print('total india salary spend is ',sum(us))
    print(f"""total india,us salary spend is ,{sum(india),sum(us)}""")
    file_content=f"""total india,us salary spend is ,{sum(india),sum(us)}"""
    if key=='employee3.csv':
        s3 = boto3.client('s3') 
        s3.put_object(Body=file_content, Bucket=bucket, Key='agg')
    
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }